In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hm-land-registry/uk-housing-prices-paid")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'uk-housing-prices-paid' dataset.
Path to dataset files: /kaggle/input/uk-housing-prices-paid


In [7]:
from glob import glob

file_path = glob(path+'/*')[0]

In [8]:
%%time
import pandas as pd

df = pd.read_csv(file_path)
df.head()

CPU times: user 39.6 s, sys: 8.39 s, total: 48 s
Wall time: 57.8 s


,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


In [9]:
%%time
import cudf as cpd

cdf = cpd.read_csv(file_path)
cdf.head()

CPU times: user 4.03 s, sys: 1.44 s, total: 5.47 s
Wall time: 8.63 s


,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


In [10]:
%%time

df['County'].unique()[:10]

CPU times: user 1.28 s, sys: 82.2 ms, total: 1.36 s
Wall time: 1.36 s


array(['GREATER MANCHESTER', 'THURROCK', 'SOMERSET', 'BEDFORDSHIRE',
       'WEST YORKSHIRE', 'WILTSHIRE', 'OXFORDSHIRE', 'CORNWALL',
       'GREATER LONDON', 'MID GLAMORGAN'], dtype=object)

In [11]:
%%time

cdf['County'].unique()[:10]

CPU times: user 51.8 ms, sys: 10.2 ms, total: 62 ms
Wall time: 126 ms


0    GREATER MANCHESTER
1              THURROCK
2              SOMERSET
3          BEDFORDSHIRE
4        WEST YORKSHIRE
5             WILTSHIRE
6           OXFORDSHIRE
7              CORNWALL
8        GREATER LONDON
9         MID GLAMORGAN
Name: County, dtype: object

In [12]:
%%time

counties = df[['County','Price']].groupby(['County'])
sum_prices = counties.sum()
df_price = sum_prices.sort_values('Price')
print(df_price[-5:])

                       Price
County                      
HERTFORDSHIRE   119710817944
ESSEX           120749211594
HAMPSHIRE       125002727729
SURREY          157580219117
GREATER LONDON  939364614777
CPU times: user 2.23 s, sys: 431 ms, total: 2.67 s
Wall time: 3.36 s


In [13]:
%%time

counties = cdf[['County','Price']].groupby(['County'])
sum_prices = counties.sum()
df_price = sum_prices.sort_values('Price')
print(df_price[-5:])

                       Price
County                      
HERTFORDSHIRE   119710817944
ESSEX           120749211594
HAMPSHIRE       125002727729
SURREY          157580219117
GREATER LONDON  939364614777
CPU times: user 101 ms, sys: 17.4 ms, total: 118 ms
Wall time: 221 ms


In [17]:
%%time

greater_london = df.loc[df['County'] == 'GREATER LONDON']
print(f'{greater_london.shape[0]} residents')

2993422 residents
CPU times: user 2.23 s, sys: 156 ms, total: 2.39 s
Wall time: 2.43 s


In [18]:
%%time

c_greater_london = cdf.loc[df['County'] == 'GREATER LONDON']
print(f'{c_greater_london.shape[0]} residents')

2993422 residents
CPU times: user 1.94 s, sys: 5.54 ms, total: 1.94 s
Wall time: 2.13 s


In [20]:
%%time
greater_london['Town/City'].unique()[:10]

CPU times: user 229 ms, sys: 816 µs, total: 230 ms
Wall time: 243 ms


array(['GREENFORD', 'ENFIELD', 'LONDON', 'ORPINGTON', 'ERITH', 'BROMLEY',
       'WOODFORD GREEN', 'UXBRIDGE', 'CARSHALTON', 'SOUTH CROYDON'],
      dtype=object)

In [21]:
%%time
c_greater_london['Town/City'].unique()[:10]

CPU times: user 5.42 ms, sys: 5.07 ms, total: 10.5 ms
Wall time: 11.2 ms


0         GREENFORD
1           ENFIELD
2            LONDON
3         ORPINGTON
4             ERITH
5           BROMLEY
6    WOODFORD GREEN
7          UXBRIDGE
8        CARSHALTON
9     SOUTH CROYDON
Name: Town/City, dtype: object

In [23]:
%%time
town_groups = df[['Town/City', 'Price']].groupby('Town/City')
town_prices = town_groups['Price'].mean()
town_counts = town_groups['Price'].count()

towns = pd.DataFrame()
towns['mean_price'] = town_prices
towns['count'] = town_counts

towns = towns.sort_values('count', ascending=False)
towns.to_csv('./towns.csv')
towns.to_parquet('./towns.parquet')
towns.iloc[:3]

CPU times: user 3.2 s, sys: 528 ms, total: 3.72 s
Wall time: 5.51 s


,mean_price,count
Town/City,,
LONDON,367955.188441,1784194
MANCHESTER,115333.803402,366133
BRISTOL,174213.722763,344839


In [24]:
%%time
c_town_groups = cdf[['Town/City', 'Price']].groupby('Town/City')
c_town_prices = c_town_groups['Price'].mean()
c_town_counts = c_town_groups['Price'].count()

c_towns = cpd.DataFrame()
c_towns['mean_price'] = c_town_prices
c_towns['count'] = c_town_counts

c_towns = c_towns.sort_values('count', ascending=False)
c_towns.to_csv('./towns.csv')
c_towns.to_parquet('./towns.parquet')
c_towns.iloc[:3]

CPU times: user 103 ms, sys: 16.1 ms, total: 119 ms
Wall time: 309 ms


,mean_price,count
Town/City,,
LONDON,367955.188441,1784194
MANCHESTER,115333.803402,366133
BRISTOL,174213.722763,344839


In [25]:
del towns
del c_towns

In [27]:
%%time
towns = pd.read_parquet('./towns.parquet')
towns.iloc[:3]

CPU times: user 25.5 ms, sys: 2.88 ms, total: 28.4 ms
Wall time: 77.1 ms


,mean_price,count
Town/City,,
LONDON,367955.188441,1784194
MANCHESTER,115333.803402,366133
BRISTOL,174213.722763,344839


In [28]:
%%time
towns = pd.read_csv('./towns.csv')
towns.iloc[:3]

CPU times: user 3.34 ms, sys: 823 µs, total: 4.17 ms
Wall time: 5.85 ms


,Town/City,mean_price,count
0,LONDON,367955.1884,1784194
1,MANCHESTER,115333.8034,366133
2,BRISTOL,174213.7228,344839


In [29]:
%%time
c_towns = cpd.read_parquet('./towns.parquet')
c_towns.iloc[:3]

CPU times: user 17.5 ms, sys: 11.7 ms, total: 29.2 ms
Wall time: 88.5 ms


,mean_price,count
Town/City,,
LONDON,367955.188441,1784194
MANCHESTER,115333.803402,366133
BRISTOL,174213.722763,344839


In [30]:
%%time
c_towns = cpd.read_csv('./towns.csv')
c_towns.iloc[:3]

CPU times: user 5.13 ms, sys: 28 µs, total: 5.16 ms
Wall time: 5.19 ms


,Town/City,mean_price,count
0,LONDON,367955.1884,1784194
1,MANCHESTER,115333.8034,366133
2,BRISTOL,174213.7228,344839
